In [ ]:
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sci
import os
%matplotlib inline
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.callbacks import ModelCheckpoint
#from biosppy.signals import ecg
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import pandas as pd
import scipy.io as sio
from os import listdir
from os.path import isfile, join
import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Dense, Activation, BatchNormalization, PReLU, Dropout, Conv2D, MaxPooling2D, Flatten, LSTM, Reshape, Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras import regularizers
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow as tf
import tensorflow.keras as keras
import keras.layers as layers
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf
import tensorflow.keras as keras
import keras.layers as layers

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
df1=pd.read_csv('/content/drive/My Drive/Motor_Vehicle_Collisions_Crashes.csv')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df1 = df1.drop(columns=['Unnamed: 0'])

In [ ]:
df1.drop_duplicates(subset=['collision_id'],inplace=True)
df1.dropna(subset=['collision_id', 'crash_date', 'crash_time'],inplace=True)
df1['crash_date'] = pd.to_datetime(df1['crash_date'])
df1['crash_time'] = pd.to_datetime(df1['crash_time'])

In [ ]:
df1['Date'] = df1['crash_date'].dt.date
df1['Month'] = df1['crash_date'].dt.month
df1['Year'] = df1['crash_date'].dt.year
df1['DOW'] = df1['crash_date'].dt.dayofweek
df1['Hour'] = df1['crash_time'].dt.hour

In [ ]:
df1['day'] = df1['crash_date'].dt.dayofyear

In [ ]:
df1['number_of_persons_injured'] = df1['number_of_persons_injured'].fillna(0)
df1['number_of_persons_killed'] = df1['number_of_persons_killed'].fillna(0)
df1['number_of_pedestrians_injured'] = df1['number_of_pedestrians_injured'].fillna(0)
df1['number_of_pedestrians_killed'] = df1['number_of_pedestrians_killed'].fillna(0)
df1['number_of_cyclist_injured'] = df1['number_of_cyclist_injured'].fillna(0)
df1['number_of_cyclist_killed'] = df1['number_of_cyclist_killed'].fillna(0)
df1['number_of_motorist_injured'] = df1['number_of_motorist_injured'].fillna(0)
df1['number_of_motorist_killed'] = df1['number_of_motorist_killed'].fillna(0)

In [ ]:
df1.head()

,crash_date,crash_time,on_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,...,vehicle_type_code_3,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5,Date,Month,Year,DOW,Hour,day
0,2021-04-14,2022-04-24 05:32:00,BRONX WHITESTONE BRIDGE,0.0,0.0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,2021-04-14,4,2021,2,5,104
1,2021-04-13,2022-04-24 21:35:00,NaN,1.0,0.0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,2021-04-13,4,2021,1,21,103
2,2021-04-15,2022-04-24 16:15:00,HUTCHINSON RIVER PARKWAY,0.0,0.0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,2021-04-15,4,2021,3,16,105
3,2021-04-13,2022-04-24 16:00:00,VANDERVORT AVENUE,0.0,0.0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,2021-04-13,4,2021,1,16,103
4,2021-04-12,2022-04-24 08:25:00,EDSON AVENUE,0.0,0.0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,2021-04-12,4,2021,0,8,102


In [ ]:
df1.shape

(1878124, 35)

In [ ]:
df1.columns

Index(['crash_date', 'crash_time', 'on_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
       'collision_id', 'vehicle_type_code1', 'vehicle_type_code2', 'borough',
       'zip_code', 'latitude', 'longitude', 'location', 'cross_street_name',
       'off_street_name', 'contributing_factor_vehicle_3',
       'contributing_factor_vehicle_4', 'vehicle_type_code_3',
       'vehicle_type_code_4', 'contributing_factor_vehicle_5',
       'vehicle_type_code_5', 'Date', 'Month', 'Year', 'DOW', 'Hour', 'day'],
      dtype='object')

In [ ]:
data = df1.copy()
data = data.fillna(0)

Predict the number of persons injured

In [ ]:
#df1.vehicle_type_code_4.unique()
X = data[['collision_id', 'zip_code', 'latitude', 'longitude','Month', 'Year', 'DOW', 'Hour', 'day']]
y = data[['number_of_persons_injured']]

In [ ]:
#X.dtypes
#X['zip_code']= X['zip_code'].astype(int64)
X['zip_code'] = pd.to_numeric(X['zip_code'],errors='coerce')
#X.zip_code.unique

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=999)

In [ ]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
y_train= np.ravel(y_train)
y_test= np.ravel(y_test)

In [ ]:
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(y_test.shape))

X_train shape: (1502499, 9)
Y_train shape: (1502499,)
X_test shape: (375625, 9)
Y_test shape: (375625,)


In [ ]:
num_classes = 44

# Scale images to the [0, 1] range
#X_train = X_train.astype("float32") / 255
#X_test = X_test.astype("float32") / 255
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
print("x_train shape:", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_test.shape[0], "test samples")

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("y_train shape:", y_train.shape)

x_train shape: (1502499, 9, 1)
1502499 train samples
375625 test samples
y_train shape: (1502499, 44)


In [ ]:
def CNN(input_shape=(9,1), classes=44): 
    X_input = Input(input_shape)
    
    X = Conv1D(filters=32,kernel_size=2, padding='same',activation='relu',name='conv1')(X_input)
    
    X = Conv1D(filters=64,kernel_size=2, padding='same',activation='relu',name='conv2')(X)
    X = MaxPooling1D(pool_size=2, name='pool1')(X)
 
    X = Conv1D(filters=128,kernel_size=2,padding='same',activation='relu',name='conv3')(X)
    X = MaxPooling1D(pool_size=2, name='pool2')(X)
    
    X = Flatten(name='flatten')(X)
    X = Dropout(0.5)(X)
    X = Dense(128,activation='relu',name='fc1')(X)
    X = Dropout(0.5)(X)
    X = Dense(classes, activation='softmax', name='fc2')(X)
    
    model = Model(inputs=X_input, outputs=X, name='CNN')
    return model
model = CNN(input_shape = (9,1), classes = 44)
model.summary()


Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9, 1)]            0         
                                                                 
 conv1 (Conv1D)              (None, 9, 32)             96        
                                                                 
 conv2 (Conv1D)              (None, 9, 64)             4160      
                                                                 
 pool1 (MaxPooling1D)        (None, 4, 64)             0         
                                                                 
 conv3 (Conv1D)              (None, 4, 128)            16512     
                                                                 
 pool2 (MaxPooling1D)        (None, 2, 128)            0         
                                                                 
 flatten (Flatten)           (None, 256)               0       

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()

In [ ]:
import time
start = time.time()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = LossHistory() 

model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, 
            validation_data=(X_test, y_test),callbacks=[history])

preds_train = model.evaluate(X_train, y_train)
print("Train Loss = " + str(preds_train[0]))
print("Train Accuracy = " + str(preds_train[1]))

preds_test  = model.evaluate(X_test, y_test)
print("Test Loss = " + str(preds_test[0]))
print("Test Accuracy = " + str(preds_test[1]))


end = time.time()
print("time:",end-start)

Epoch 1/10
11739/11739 [==============================] - 110s 9ms/step - loss: nan - accuracy: 0.7848 - val_loss: nan - val_accuracy: 0.7895
Epoch 2/10
11739/11739 [==============================] - 111s 9ms/step - loss: nan - accuracy: 0.7902 - val_loss: nan - val_accuracy: 0.7895
Epoch 3/10
11739/11739 [==============================] - 111s 9ms/step - loss: nan - accuracy: 0.7902 - val_loss: nan - val_accuracy: 0.7895
Epoch 4/10
11739/11739 [==============================] - 118s 10ms/step - loss: nan - accuracy: 0.7902 - val_loss: nan - val_accuracy: 0.7895
Epoch 5/10
11739/11739 [==============================] - 109s 9ms/step - loss: nan - accuracy: 0.7902 - val_loss: nan - val_accuracy: 0.7895
Epoch 6/10
11739/11739 [==============================] - 115s 10ms/step - loss: nan - accuracy: 0.7902 - val_loss: nan - val_accuracy: 0.7895
Epoch 7/10
11739/11739 [==============================] - 115s 10ms/step - loss: nan - accuracy: 0.7902 - val_loss: nan - val_accuracy: 0.7895
Epo

predict the number of persons killed

In [ ]:
X = data[['collision_id', 'zip_code', 'latitude', 'longitude','Month', 'Year', 'DOW', 'Hour', 'day']]
y = data[['number_of_persons_killed']]

In [ ]:
X['zip_code'] = pd.to_numeric(X['zip_code'],errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=999)

In [ ]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_train= np.ravel(y_train)
y_test= np.ravel(y_test)

In [ ]:
num_classes = 9

# Scale images to the [0, 1] range
#X_train = X_train.astype("float32") / 255
#X_test = X_test.astype("float32") / 255
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
print("x_train shape:", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("y_train shape:", y_train.shape)

x_train shape: (1502499, 9, 1)
1502499 train samples
375625 test samples
y_train shape: (1502499, 9)


In [ ]:
def CNN(input_shape=(9,1), classes=9): 
    X_input = Input(input_shape)
    
    X = Conv1D(filters=32,kernel_size=2, padding='same',activation='relu',name='conv1')(X_input)
    
    X = Conv1D(filters=64,kernel_size=2, padding='same',activation='relu',name='conv2')(X)
    X = MaxPooling1D(pool_size=2, name='pool1')(X)
 
    X = Conv1D(filters=128,kernel_size=2,padding='same',activation='relu',name='conv3')(X)
    X = MaxPooling1D(pool_size=2, name='pool2')(X)
    
    X = Flatten(name='flatten')(X)
    X = Dropout(0.5)(X)
    X = Dense(128,activation='relu',name='fc1')(X)
    X = Dropout(0.5)(X)
    X = Dense(classes, activation='softmax', name='fc2')(X)
    
    model = Model(inputs=X_input, outputs=X, name='CNN')
    return model
model = CNN(input_shape = (9,1), classes = 9)
model.summary()

Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 9, 1)]            0         
                                                                 
 conv1 (Conv1D)              (None, 9, 32)             96        
                                                                 
 conv2 (Conv1D)              (None, 9, 64)             4160      
                                                                 
 pool1 (MaxPooling1D)        (None, 4, 64)             0         
                                                                 
 conv3 (Conv1D)              (None, 4, 128)            16512     
                                                                 
 pool2 (MaxPooling1D)        (None, 2, 128)            0         
                                                                 
 flatten (Flatten)           (None, 256)               0       

In [ ]:
import time
start = time.time()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = LossHistory() 

model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, 
            validation_data=(X_test, y_test),callbacks=[history])

preds_train = model.evaluate(X_train, y_train)
print("Train Loss = " + str(preds_train[0]))
print("Train Accuracy = " + str(preds_train[1]))

preds_test  = model.evaluate(X_test, y_test)
print("Test Loss = " + str(preds_test[0]))
print("Test Accuracy = " + str(preds_test[1]))


end = time.time()
print("time:",end-start)

Epoch 1/10
11739/11739 [==============================] - 114s 10ms/step - loss: nan - accuracy: 0.9983 - val_loss: nan - val_accuracy: 0.9988
Epoch 2/10
11739/11739 [==============================] - 111s 9ms/step - loss: nan - accuracy: 0.9987 - val_loss: nan - val_accuracy: 0.9988
Epoch 3/10
11739/11739 [==============================] - 116s 10ms/step - loss: nan - accuracy: 0.9987 - val_loss: nan - val_accuracy: 0.9988
Epoch 4/10
11739/11739 [==============================] - 112s 10ms/step - loss: nan - accuracy: 0.9987 - val_loss: nan - val_accuracy: 0.9988
Epoch 5/10
11739/11739 [==============================] - 113s 10ms/step - loss: nan - accuracy: 0.9987 - val_loss: nan - val_accuracy: 0.9988
Epoch 6/10
11739/11739 [==============================] - 112s 10ms/step - loss: nan - accuracy: 0.9987 - val_loss: nan - val_accuracy: 0.9988
Epoch 7/10
11739/11739 [==============================] - 112s 10ms/step - loss: nan - accuracy: 0.9987 - val_loss: nan - val_accuracy: 0.9988


predict the number of pedestrians injured

In [ ]:
X = data[['collision_id', 'zip_code', 'latitude', 'longitude','Month', 'Year', 'DOW', 'Hour', 'day']]
y = data[['number_of_pedestrians_injured']]
X['zip_code'] = pd.to_numeric(X['zip_code'],errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=999)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_train= np.ravel(y_train)
y_test= np.ravel(y_test)

In [ ]:
num_classes = 28

# Scale images to the [0, 1] range
#X_train = X_train.astype("float32") / 255
#X_test = X_test.astype("float32") / 255
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
print("x_train shape:", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("y_train shape:", y_train.shape)

x_train shape: (1502499, 9, 1)
1502499 train samples
375625 test samples
y_train shape: (1502499, 28)


In [ ]:
def CNN(input_shape=(9,1), classes=28): 
    X_input = Input(input_shape)
    
    X = Conv1D(filters=32,kernel_size=2, padding='same',activation='relu',name='conv1')(X_input)
    
    X = Conv1D(filters=64,kernel_size=2, padding='same',activation='relu',name='conv2')(X)
    X = MaxPooling1D(pool_size=2, name='pool1')(X)
 
    X = Conv1D(filters=128,kernel_size=2,padding='same',activation='relu',name='conv3')(X)
    X = MaxPooling1D(pool_size=2, name='pool2')(X)
    
    X = Flatten(name='flatten')(X)
    X = Dropout(0.5)(X)
    X = Dense(128,activation='relu',name='fc1')(X)
    X = Dropout(0.5)(X)
    X = Dense(classes, activation='softmax', name='fc2')(X)
    
    model = Model(inputs=X_input, outputs=X, name='CNN')
    return model
model = CNN(input_shape = (9,1), classes = 28)
model.summary()

Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 9, 1)]            0         
                                                                 
 conv1 (Conv1D)              (None, 9, 32)             96        
                                                                 
 conv2 (Conv1D)              (None, 9, 64)             4160      
                                                                 
 pool1 (MaxPooling1D)        (None, 4, 64)             0         
                                                                 
 conv3 (Conv1D)              (None, 4, 128)            16512     
                                                                 
 pool2 (MaxPooling1D)        (None, 2, 128)            0         
                                                                 
 flatten (Flatten)           (None, 256)               0       

In [ ]:
import time
start = time.time()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = LossHistory() 

model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, 
            validation_data=(X_test, y_test),callbacks=[history])

preds_train = model.evaluate(X_train, y_train)
print("Train Loss = " + str(preds_train[0]))
print("Train Accuracy = " + str(preds_train[1]))

preds_test  = model.evaluate(X_test, y_test)
print("Test Loss = " + str(preds_test[0]))
print("Test Accuracy = " + str(preds_test[1]))


end = time.time()
print("time:",end-start)

Epoch 1/10
11739/11739 [==============================] - 110s 9ms/step - loss: nan - accuracy: 0.9444 - val_loss: nan - val_accuracy: 0.9489
Epoch 2/10
11739/11739 [==============================] - 107s 9ms/step - loss: nan - accuracy: 0.9495 - val_loss: nan - val_accuracy: 0.9489
Epoch 3/10
11739/11739 [==============================] - 110s 9ms/step - loss: nan - accuracy: 0.9495 - val_loss: nan - val_accuracy: 0.9489
Epoch 4/10
11739/11739 [==============================] - 108s 9ms/step - loss: nan - accuracy: 0.9495 - val_loss: nan - val_accuracy: 0.9489
Epoch 5/10
11739/11739 [==============================] - 113s 10ms/step - loss: nan - accuracy: 0.9495 - val_loss: nan - val_accuracy: 0.9489
Epoch 6/10
11739/11739 [==============================] - 115s 10ms/step - loss: nan - accuracy: 0.9495 - val_loss: nan - val_accuracy: 0.9489
Epoch 7/10
11739/11739 [==============================] - 112s 10ms/step - loss: nan - accuracy: 0.9495 - val_loss: nan - val_accuracy: 0.9489
Epo

predict the number of pedestrians killed

In [ ]:
X = data[['collision_id', 'zip_code', 'latitude', 'longitude','Month', 'Year', 'DOW', 'Hour', 'day']]
y = data[['number_of_pedestrians_killed']]
X['zip_code'] = pd.to_numeric(X['zip_code'],errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=999)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_train= np.ravel(y_train)
y_test= np.ravel(y_test)

In [ ]:
num_classes = 7

# Scale images to the [0, 1] range
#X_train = X_train.astype("float32") / 255
#X_test = X_test.astype("float32") / 255
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
print("x_train shape:", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_test.shape[0], "test samples")
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("y_train shape:", y_train.shape)

x_train shape: (1502499, 9, 1)
1502499 train samples
375625 test samples
y_train shape: (1502499, 7)


In [ ]:
def CNN(input_shape=(9,1), classes=7): 
    X_input = Input(input_shape)
    
    X = Conv1D(filters=32,kernel_size=2, padding='same',activation='relu',name='conv1')(X_input)
    
    X = Conv1D(filters=64,kernel_size=2, padding='same',activation='relu',name='conv2')(X)
    X = MaxPooling1D(pool_size=2, name='pool1')(X)
 
    X = Conv1D(filters=128,kernel_size=2,padding='same',activation='relu',name='conv3')(X)
    X = MaxPooling1D(pool_size=2, name='pool2')(X)
    
    X = Flatten(name='flatten')(X)
    X = Dropout(0.5)(X)
    X = Dense(128,activation='relu',name='fc1')(X)
    X = Dropout(0.5)(X)
    X = Dense(classes, activation='softmax', name='fc2')(X)
    
    model = Model(inputs=X_input, outputs=X, name='CNN')
    return model
model = CNN(input_shape = (9,1), classes = 7)
model.summary()

Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 9, 1)]            0         
                                                                 
 conv1 (Conv1D)              (None, 9, 32)             96        
                                                                 
 conv2 (Conv1D)              (None, 9, 64)             4160      
                                                                 
 pool1 (MaxPooling1D)        (None, 4, 64)             0         
                                                                 
 conv3 (Conv1D)              (None, 4, 128)            16512     
                                                                 
 pool2 (MaxPooling1D)        (None, 2, 128)            0         
                                                                 
 flatten (Flatten)           (None, 256)               0       

In [ ]:
import time
start = time.time()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = LossHistory() 

model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, 
            validation_data=(X_test, y_test),callbacks=[history])

preds_train = model.evaluate(X_train, y_train)
print("Train Loss = " + str(preds_train[0]))
print("Train Accuracy = " + str(preds_train[1]))

preds_test  = model.evaluate(X_test, y_test)
print("Test Loss = " + str(preds_test[0]))
print("Test Accuracy = " + str(preds_test[1]))


end = time.time()
print("time:",end-start)

Epoch 1/10
11739/11739 [==============================] - 108s 9ms/step - loss: nan - accuracy: 0.9988 - val_loss: nan - val_accuracy: 0.9993
Epoch 2/10
11739/11739 [==============================] - 110s 9ms/step - loss: nan - accuracy: 0.9993 - val_loss: nan - val_accuracy: 0.9993
Epoch 3/10
11739/11739 [==============================] - 112s 10ms/step - loss: nan - accuracy: 0.9993 - val_loss: nan - val_accuracy: 0.9993
Epoch 4/10
11739/11739 [==============================] - 109s 9ms/step - loss: nan - accuracy: 0.9993 - val_loss: nan - val_accuracy: 0.9993
Epoch 5/10
11739/11739 [==============================] - 114s 10ms/step - loss: nan - accuracy: 0.9993 - val_loss: nan - val_accuracy: 0.9993
Epoch 6/10
11739/11739 [==============================] - 110s 9ms/step - loss: nan - accuracy: 0.9993 - val_loss: nan - val_accuracy: 0.9993
Epoch 7/10
11739/11739 [==============================] - 113s 10ms/step - loss: nan - accuracy: 0.9993 - val_loss: nan - val_accuracy: 0.9993
Epo

predict the number of cyclist injured

In [ ]:
X = data[['collision_id', 'zip_code', 'latitude', 'longitude','Month', 'Year', 'DOW', 'Hour', 'day']]
y = data[['number_of_cyclist_injured']]
X['zip_code'] = pd.to_numeric(X['zip_code'],errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=999)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_train= np.ravel(y_train)
y_test= np.ravel(y_test)

In [ ]:
num_classes = 5

# Scale images to the [0, 1] range
#X_train = X_train.astype("float32") / 255
#X_test = X_test.astype("float32") / 255
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
print("x_train shape:", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_test.shape[0], "test samples")
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("y_train shape:", y_train.shape)

x_train shape: (1502499, 9, 1)
1502499 train samples
375625 test samples
y_train shape: (1502499, 5)


In [ ]:
def CNN(input_shape=(9,1), classes=5): 
    X_input = Input(input_shape)
    
    X = Conv1D(filters=32,kernel_size=2, padding='same',activation='relu',name='conv1')(X_input)
    
    X = Conv1D(filters=64,kernel_size=2, padding='same',activation='relu',name='conv2')(X)
    X = MaxPooling1D(pool_size=2, name='pool1')(X)
 
    X = Conv1D(filters=128,kernel_size=2,padding='same',activation='relu',name='conv3')(X)
    X = MaxPooling1D(pool_size=2, name='pool2')(X)
    
    X = Flatten(name='flatten')(X)
    X = Dropout(0.5)(X)
    X = Dense(128,activation='relu',name='fc1')(X)
    X = Dropout(0.5)(X)
    X = Dense(classes, activation='softmax', name='fc2')(X)
    
    model = Model(inputs=X_input, outputs=X, name='CNN')
    return model
model = CNN(input_shape = (9,1), classes = 5)
model.summary()

Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 9, 1)]            0         
                                                                 
 conv1 (Conv1D)              (None, 9, 32)             96        
                                                                 
 conv2 (Conv1D)              (None, 9, 64)             4160      
                                                                 
 pool1 (MaxPooling1D)        (None, 4, 64)             0         
                                                                 
 conv3 (Conv1D)              (None, 4, 128)            16512     
                                                                 
 pool2 (MaxPooling1D)        (None, 2, 128)            0         
                                                                 
 flatten (Flatten)           (None, 256)               0       

In [ ]:
import time
start = time.time()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = LossHistory() 

model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, 
            validation_data=(X_test, y_test),callbacks=[history])

preds_train = model.evaluate(X_train, y_train)
print("Train Loss = " + str(preds_train[0]))
print("Train Accuracy = " + str(preds_train[1]))

preds_test  = model.evaluate(X_test, y_test)
print("Test Loss = " + str(preds_test[0]))
print("Test Accuracy = " + str(preds_test[1]))


end = time.time()
print("time:",end-start)

Epoch 1/10
11739/11739 [==============================] - 111s 9ms/step - loss: nan - accuracy: 0.9752 - val_loss: nan - val_accuracy: 0.9761
Epoch 2/10
11739/11739 [==============================] - 112s 10ms/step - loss: nan - accuracy: 0.9761 - val_loss: nan - val_accuracy: 0.9761
Epoch 3/10
11739/11739 [==============================] - 107s 9ms/step - loss: nan - accuracy: 0.9761 - val_loss: nan - val_accuracy: 0.9761
Epoch 4/10
11739/11739 [==============================] - 107s 9ms/step - loss: nan - accuracy: 0.9761 - val_loss: nan - val_accuracy: 0.9761
Epoch 5/10
11739/11739 [==============================] - 110s 9ms/step - loss: nan - accuracy: 0.9761 - val_loss: nan - val_accuracy: 0.9761
Epoch 6/10
11739/11739 [==============================] - 107s 9ms/step - loss: nan - accuracy: 0.9761 - val_loss: nan - val_accuracy: 0.9761
Epoch 7/10
11739/11739 [==============================] - 111s 9ms/step - loss: nan - accuracy: 0.9761 - val_loss: nan - val_accuracy: 0.9761
Epoch

predict the number of cyclist killed

In [ ]:
X = data[['collision_id', 'zip_code', 'latitude', 'longitude','Month', 'Year', 'DOW', 'Hour', 'day']]
y = data[['number_of_cyclist_killed']]
X['zip_code'] = pd.to_numeric(X['zip_code'],errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=999)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_train= np.ravel(y_train)
y_test= np.ravel(y_test)

In [ ]:
num_classes = 3

# Scale images to the [0, 1] range
#X_train = X_train.astype("float32") / 255
#X_test = X_test.astype("float32") / 255
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
print("x_train shape:", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_test.shape[0], "test samples")
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("y_train shape:", y_train.shape)

x_train shape: (1502499, 9, 1)
1502499 train samples
375625 test samples
y_train shape: (1502499, 3)


In [ ]:
def CNN(input_shape=(9,1), classes=3): 
    X_input = Input(input_shape)
    
    X = Conv1D(filters=32,kernel_size=2, padding='same',activation='relu',name='conv1')(X_input)
    
    X = Conv1D(filters=64,kernel_size=2, padding='same',activation='relu',name='conv2')(X)
    X = MaxPooling1D(pool_size=2, name='pool1')(X)
 
    X = Conv1D(filters=128,kernel_size=2,padding='same',activation='relu',name='conv3')(X)
    X = MaxPooling1D(pool_size=2, name='pool2')(X)
    
    X = Flatten(name='flatten')(X)
    X = Dropout(0.5)(X)
    X = Dense(128,activation='relu',name='fc1')(X)
    X = Dropout(0.5)(X)
    X = Dense(classes, activation='softmax', name='fc2')(X)
    
    model = Model(inputs=X_input, outputs=X, name='CNN')
    return model
model = CNN(input_shape = (9,1), classes = 3)
model.summary()

Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 9, 1)]            0         
                                                                 
 conv1 (Conv1D)              (None, 9, 32)             96        
                                                                 
 conv2 (Conv1D)              (None, 9, 64)             4160      
                                                                 
 pool1 (MaxPooling1D)        (None, 4, 64)             0         
                                                                 
 conv3 (Conv1D)              (None, 4, 128)            16512     
                                                                 
 pool2 (MaxPooling1D)        (None, 2, 128)            0         
                                                                 
 flatten (Flatten)           (None, 256)               0       

In [ ]:
import time
start = time.time()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = LossHistory() 

model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, 
            validation_data=(X_test, y_test),callbacks=[history])

preds_train = model.evaluate(X_train, y_train)
print("Train Loss = " + str(preds_train[0]))
print("Train Accuracy = " + str(preds_train[1]))

preds_test  = model.evaluate(X_test, y_test)
print("Test Loss = " + str(preds_test[0]))
print("Test Accuracy = " + str(preds_test[1]))


end = time.time()
print("time:",end-start)

Epoch 1/10
11739/11739 [==============================] - 106s 9ms/step - loss: nan - accuracy: 0.9997 - val_loss: nan - val_accuracy: 0.9999
Epoch 2/10
11739/11739 [==============================] - 106s 9ms/step - loss: nan - accuracy: 0.9999 - val_loss: nan - val_accuracy: 0.9999
Epoch 3/10
11739/11739 [==============================] - 108s 9ms/step - loss: nan - accuracy: 0.9999 - val_loss: nan - val_accuracy: 0.9999
Epoch 4/10
11739/11739 [==============================] - 113s 10ms/step - loss: nan - accuracy: 0.9999 - val_loss: nan - val_accuracy: 0.9999
Epoch 5/10
11739/11739 [==============================] - 106s 9ms/step - loss: nan - accuracy: 0.9999 - val_loss: nan - val_accuracy: 0.9999
Epoch 6/10
11739/11739 [==============================] - 110s 9ms/step - loss: nan - accuracy: 0.9999 - val_loss: nan - val_accuracy: 0.9999
Epoch 7/10
11739/11739 [==============================] - 116s 10ms/step - loss: nan - accuracy: 0.9999 - val_loss: nan - val_accuracy: 0.9999
Epoc

predict the number of motorist injured

In [ ]:
X = data[['collision_id', 'zip_code', 'latitude', 'longitude','Month', 'Year', 'DOW', 'Hour', 'day']]
y = data[['number_of_motorist_injured']]
X['zip_code'] = pd.to_numeric(X['zip_code'],errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=999)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_train= np.ravel(y_train)
y_test= np.ravel(y_test)

In [ ]:
num_classes = 44

# Scale images to the [0, 1] range
#X_train = X_train.astype("float32") / 255
#X_test = X_test.astype("float32") / 255
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
print("x_train shape:", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_test.shape[0], "test samples")
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("y_train shape:", y_train.shape)

x_train shape: (1502499, 9, 1)
1502499 train samples
375625 test samples
y_train shape: (1502499, 44)


In [ ]:
def CNN(input_shape=(9,1), classes=44): 
    X_input = Input(input_shape)
    
    X = Conv1D(filters=32,kernel_size=2, padding='same',activation='relu',name='conv1')(X_input)
    
    X = Conv1D(filters=64,kernel_size=2, padding='same',activation='relu',name='conv2')(X)
    X = MaxPooling1D(pool_size=2, name='pool1')(X)
 
    X = Conv1D(filters=128,kernel_size=2,padding='same',activation='relu',name='conv3')(X)
    X = MaxPooling1D(pool_size=2, name='pool2')(X)
    
    X = Flatten(name='flatten')(X)
    X = Dropout(0.5)(X)
    X = Dense(128,activation='relu',name='fc1')(X)
    X = Dropout(0.5)(X)
    X = Dense(classes, activation='softmax', name='fc2')(X)
    
    model = Model(inputs=X_input, outputs=X, name='CNN')
    return model
model = CNN(input_shape = (9,1), classes = 44)
model.summary()

Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 9, 1)]            0         
                                                                 
 conv1 (Conv1D)              (None, 9, 32)             96        
                                                                 
 conv2 (Conv1D)              (None, 9, 64)             4160      
                                                                 
 pool1 (MaxPooling1D)        (None, 4, 64)             0         
                                                                 
 conv3 (Conv1D)              (None, 4, 128)            16512     
                                                                 
 pool2 (MaxPooling1D)        (None, 2, 128)            0         
                                                                 
 flatten (Flatten)           (None, 256)               0       

In [ ]:
import time
start = time.time()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = LossHistory() 

model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, 
            validation_data=(X_test, y_test),callbacks=[history])

preds_train = model.evaluate(X_train, y_train)
print("Train Loss = " + str(preds_train[0]))
print("Train Accuracy = " + str(preds_train[1]))

preds_test  = model.evaluate(X_test, y_test)
print("Test Loss = " + str(preds_test[0]))
print("Test Accuracy = " + str(preds_test[1]))


end = time.time()
print("time:",end-start)

Epoch 1/10
11739/11739 [==============================] - 110s 9ms/step - loss: nan - accuracy: 0.8636 - val_loss: nan - val_accuracy: 0.8647
Epoch 2/10
11739/11739 [==============================] - 111s 9ms/step - loss: nan - accuracy: 0.8648 - val_loss: nan - val_accuracy: 0.8647
Epoch 3/10
11739/11739 [==============================] - 116s 10ms/step - loss: nan - accuracy: 0.8648 - val_loss: nan - val_accuracy: 0.8647
Epoch 4/10
11739/11739 [==============================] - 118s 10ms/step - loss: nan - accuracy: 0.8648 - val_loss: nan - val_accuracy: 0.8647
Epoch 5/10
11739/11739 [==============================] - 116s 10ms/step - loss: nan - accuracy: 0.8648 - val_loss: nan - val_accuracy: 0.8647
Epoch 6/10
11739/11739 [==============================] - 118s 10ms/step - loss: nan - accuracy: 0.8648 - val_loss: nan - val_accuracy: 0.8647
Epoch 7/10
11739/11739 [==============================] - 117s 10ms/step - loss: nan - accuracy: 0.8648 - val_loss: nan - val_accuracy: 0.8647
E

predict the number of motorist killed

In [ ]:
X = data[['collision_id', 'zip_code', 'latitude', 'longitude','Month', 'Year', 'DOW', 'Hour', 'day']]
y = data[['number_of_motorist_killed']]
X['zip_code'] = pd.to_numeric(X['zip_code'],errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=999)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_train= np.ravel(y_train)
y_test= np.ravel(y_test)

In [ ]:
num_classes = 6

# Scale images to the [0, 1] range
#X_train = X_train.astype("float32") / 255
#X_test = X_test.astype("float32") / 255
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
print("x_train shape:", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_test.shape[0], "test samples")
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("y_train shape:", y_train.shape)

x_train shape: (1502499, 9, 1)
1502499 train samples
375625 test samples
y_train shape: (1502499, 6)


In [ ]:
def CNN(input_shape=(9,1), classes=6): 
    X_input = Input(input_shape)
    
    X = Conv1D(filters=32,kernel_size=2, padding='same',activation='relu',name='conv1')(X_input)
    
    X = Conv1D(filters=64,kernel_size=2, padding='same',activation='relu',name='conv2')(X)
    X = MaxPooling1D(pool_size=2, name='pool1')(X)
 
    X = Conv1D(filters=128,kernel_size=2,padding='same',activation='relu',name='conv3')(X)
    X = MaxPooling1D(pool_size=2, name='pool2')(X)
    
    X = Flatten(name='flatten')(X)
    X = Dropout(0.5)(X)
    X = Dense(128,activation='relu',name='fc1')(X)
    X = Dropout(0.5)(X)
    X = Dense(classes, activation='softmax', name='fc2')(X)
    
    model = Model(inputs=X_input, outputs=X, name='CNN')
    return model
model = CNN(input_shape = (9,1), classes = 6)
model.summary()

Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 9, 1)]            0         
                                                                 
 conv1 (Conv1D)              (None, 9, 32)             96        
                                                                 
 conv2 (Conv1D)              (None, 9, 64)             4160      
                                                                 
 pool1 (MaxPooling1D)        (None, 4, 64)             0         
                                                                 
 conv3 (Conv1D)              (None, 4, 128)            16512     
                                                                 
 pool2 (MaxPooling1D)        (None, 2, 128)            0         
                                                                 
 flatten (Flatten)           (None, 256)               0       

In [ ]:
import time
start = time.time()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = LossHistory() 

model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, 
            validation_data=(X_test, y_test),callbacks=[history])

preds_train = model.evaluate(X_train, y_train)
print("Train Loss = " + str(preds_train[0]))
print("Train Accuracy = " + str(preds_train[1]))

preds_test  = model.evaluate(X_test, y_test)
print("Test Loss = " + str(preds_test[0]))
print("Test Accuracy = " + str(preds_test[1]))


end = time.time()
print("time:",end-start)

Epoch 1/10
11739/11739 [==============================] - 117s 10ms/step - loss: nan - accuracy: 0.9992 - val_loss: nan - val_accuracy: 0.9996
Epoch 2/10
11739/11739 [==============================] - 115s 10ms/step - loss: nan - accuracy: 0.9995 - val_loss: nan - val_accuracy: 0.9996
Epoch 3/10
11739/11739 [==============================] - 114s 10ms/step - loss: nan - accuracy: 0.9995 - val_loss: nan - val_accuracy: 0.9996
Epoch 4/10
11739/11739 [==============================] - 113s 10ms/step - loss: nan - accuracy: 0.9995 - val_loss: nan - val_accuracy: 0.9996
Epoch 5/10
11739/11739 [==============================] - 111s 9ms/step - loss: nan - accuracy: 0.9995 - val_loss: nan - val_accuracy: 0.9996
Epoch 6/10
11739/11739 [==============================] - 111s 9ms/step - loss: nan - accuracy: 0.9995 - val_loss: nan - val_accuracy: 0.9996
Epoch 7/10
11739/11739 [==============================] - 108s 9ms/step - loss: nan - accuracy: 0.9995 - val_loss: nan - val_accuracy: 0.9996
Ep